In [12]:
import spacy
import spacy_transformers

In [4]:
!python3 -m spacy download pl_core_news_sm


✘ No compatible package found for 'pl_core_news_ln' (spaCy v3.6.1)



In [17]:
nlp = spacy.load("pl_core_news_sm")
nlp

In [18]:
import json
# https://www.kaggle.com/datasets/finalepoch/medical-ner
with open('data.json', 'r') as f:
    data = json.load(f)

In [19]:
data['examples'][0]


{'id': '25aed6f4-5fd4-11ee-a1eb-662001ce32f6',
 'content': 'Czy istnieje uczelnia o nazwie Akademia Piotrkowska w Krakow?',
 'metadata': {},
 'annotations': [{'id': '25aed794-5fd4-11ee-a1eb-662001ce32f6',
   'tag_id': 'c06bd022-6ded-44a5-8d90-f17685bb85a1',
   'end': 51,
   'start': 31,
   'example_id': '25aed7c6-5fd4-11ee-a1eb-662001ce32f6',
   'tag_name': 'University',
   'value': 'Akademia Piotrkowska',
   'correct': None,
   'human_annotations': [{'timestamp': '2020-03-21T00:24:32.098000Z',
     'annotator_id': 1,
     'tagged_token_id': '979b6f5d-8e2d-4214-9e08-cc283ac3df1e',
     'name': 'Ashpat123',
     'reason': 'exploration'}],
   'model_annotations': []}],
 'classifications': []}

In [20]:
training_data = []
for example in data['examples']:
  temp_dict = {}
  temp_dict['text'] = example['content']
  temp_dict['entities'] = []
  for annotation in example['annotations']:
    start = annotation['start']
    end = annotation['end']
    label = annotation['tag_name'].upper()
    temp_dict['entities'].append((start, end, label))
  training_data.append(temp_dict)

print(training_data[0])

{'text': 'Czy istnieje uczelnia o nazwie Akademia Piotrkowska w Krakow?', 'entities': [(31, 51, 'UNIVERSITY')]}


In [21]:
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("pl") # load a new spacy model
doc_bin = DocBin()

In [22]:
from spacy.util import filter_spans

for training_example  in tqdm(training_data):
    text = training_example['text']
    labels = training_example['entities']
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in labels:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    filtered_ents = filter_spans(ents)
    doc.ents = filtered_ents
    doc_bin.add(doc)

doc_bin.to_disk("train.spacy")


100%|██████████| 278/278 [00:00<00:00, 18958.08it/s]

Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity


In [23]:
!python3 -m spacy init fill-config base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [24]:
!python3 -m spacy train config.cfg --output ./ --paths.train ./train.spacy --paths.dev ./train.spacy


ℹ Saving to output directory: .
ℹ Using CPU
ℹ To switch to GPU 0, use the option: --gpu-id 0

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     53.20    0.00    0.00    0.00    0.00
  7     200        230.18   1710.67  100.00  100.00  100.00    1.00
 16     400          0.00      0.00  100.00  100.00  100.00    1.00
 27     600          0.00      0.00  100.00  100.00  100.00    1.00
 41     800          0.00      0.00  100.00  100.00  100.00    1.00
 58    1000          0.00      0.00  100.00  100.00  100.00    1.00
 79    1200          0.00      0.00  100.00  100.00  100.00    1.00
104    1400          0.00      0.00  100.00  100.00  10

In [26]:
nlp_ner = spacy.load("model-best")

In [41]:
doc = nlp_ner("Chce byc studentem w Kraków")

spacy.displacy.render(doc, style="ent", jupyter=True)


In [28]:
doc.ents

(Polska Akademia, Gołyszu)

In [7]:
!python3 -m spacy download pl_core_news_lg

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 573.7 MB 107 kB/s  eta 0:00:01    |█████▍                          | 96.9 MB 4.1 MB/s eta 0:01:56     |██████▌                         | 117.1 MB 3.7 MB/s eta 0:02:03     |████████████                    | 215.2 MB 507 kB/s eta 0:11:47     |███████████████████▏            | 343.2 MB 3.7 MB/s eta 0:01:02     |████████████████████▎           | 362.8 MB 4.0 MB/s eta 0:00:53     |█████████████████████▎          | 382.0 MB 3.8 MB/s eta 0:00:50
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.10/bin/python3 -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the package via spacy.load('pl_core_news_lg')


In [9]:
nlp_test = spacy.load("pl_core_news_lg")

In [11]:
doc = nlp_test("Krakow")

spacy.displacy.render(doc, style="ent", jupyter=True)